In [1]:
import pycolmap as pc
from pathlib import Path
import sys
from IPython.display import clear_output

sys.path.append('/src/app')
sys.path.append('/src/')
sys.path.append('src')
from place_utils import localize_places

feature_conf = "superpoint_aachen"
matcher_conf = "superglue"

import numpy as np
import cv2
import tqdm
import os
# from IPython.display import clear_output

# Clear the output of the current cell
# clear_output()

def get_qt_vector(recon):
    res = {}
    for k,image in recon.images.items():
        qtvec = np.concatenate((image.qvec,image.tvec), axis=0)
        res[image.name] = qtvec
    return res

def matcher_localize(image_paths, dataset_name, weightNo, intrinsic,num=20, rank=5, cvtColor=None, limit=99999999):
    poses = {}
    count = 0
    for image_path in tqdm.tqdm(image_paths):
        args = {"name": dataset_name,
                "input": image_path,
                "num": num,
                "rank": rank,
                "feature_conf":feature_conf,
                "weight":weightNo,
                "matcher_conf":matcher_conf,}
        if intrinsic is not None: args['intrinsic'] = intrinsic
        print(args)
        image = cv2.imread(image_path)
        if cvtColor: image = cv2.cvtColor(image, cvtColor)
        queries = [(os.path.basename(image_path), image)]
        try:
            pose = localize_places(queries, args)
        except Exception as e:
            print(e)
            pose = "ERROR"
        poses[image_path] = pose
        count+=1
        if count == limit: break
        clear_output()
    return poses

def set_query_poses_to_model(recon, poses):
    """
    use output from matcher localize
    """
    for i, image in recon.images.items():
        image_name = image.name
        q = poses[image_name][0]['orientation']
        t = poses[image_name][0]['position']
        old_qvec = recon.images[i].qvec
        old_tvec = recon.images[i].tvec
        recon.images[i].tvec = np.array([t['x'], t['y'], t['z']])
        recon.images[i].qvec = np.array([q['w'], q['x'], q['y'], q['z']])
    print("set new camera poses success!")

from pathlib import Path
import os

def run_eval_pipeline(map_name, image_path, intrinsic, weightNo):
    txt=''
    map_path = Path("/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/") / map_name
    image_paths = [im.path for im in os.scandir(image_path)]
    poses = matcher_localize(image_paths, map_name, weightNo, intrinsic,limit=9999999)
    for k,v in poses.items():
        name = '/'.join(k.split("/")[-1:])
        if v == "ERROR":
            txt += f"{name},ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR\n"
        else:
            q = v[0]['orientation']
            t = v[0]['position']
            txt += f"{name},{t['x']},{t['y']},{t['z']},{q['w']},{q['x']},{q['y']},{q['z']}\n"
            # print(poses)
            # print([os.path.basename(i) for i in image_paths])
    return txt



Using cache found in /root/.cache/torch/hub/yxgeee_OpenIBL_master


## Read SR1 intrinsic

In [22]:
import pycolmap as pc
recon = pc.Reconstruction()
model_path = "/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/bld4_1000Hour_3/sfm_superpoint+superglue"
recon.read(model_path)
camera = recon.cameras[1]

params = [camera.model_name, camera.width, camera.height] + camera.params_to_string().split(',')
save_ = "/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/bld4_1000Hour/sr1_estimated_intrinsic.txt"
intrins_param = ' '.join(map(str, params))
with open(str(save_), 'w') as f:
    f.write(intrins_param)

## localize

In [2]:
from pathlib import Path
image_path = "/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/1000Hour_sampled"
# image_path = '/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/bld4f1-20221101/db'
map_name = "bld4f1-upper-p06top19"
weightNo = 0
intrinsic = "/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/bld4_1000Hour/sr1_estimated_intrinsic.txt"
txt = run_eval_pipeline(map_name, image_path, intrinsic, weightNo)
with open(Path(image_path) / "localize_result_upper.txt", 'w') as f:
    f.write(txt)

100%|██████████| 105/105 [11:23<00:00,  6.51s/it]


## Assign extra localized images to model

### ref model for camera intrinsic

In [2]:
import pycolmap as pc
recon = pc.Reconstruction()
model_path = "/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/bld4_1000Hour_3/sfm_superpoint+superglue"
recon.read(model_path)
sr1_camera = recon.cameras[1]
sr1_camera.camera_id = 2
sr1_camera

<Camera 'camera_id=2, model=SIMPLE_RADIAL, width=1280, height=720, num_params=4'>

In [3]:
sr1_camera

<Camera 'camera_id=2, model=SIMPLE_RADIAL, width=1280, height=720, num_params=4'>

### target model to add camera

In [4]:
recon2 = pc.Reconstruction()
model_path = "/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/bld4f1-20221101/weight0/map0"
recon2.read(model_path)
recon2.add_camera(sr1_camera)

In [5]:
recon2.cameras

{2: <Camera 'camera_id=2, model=SIMPLE_RADIAL, width=1280, height=720, num_params=4'>, 1: <Camera 'camera_id=1, model=SIMPLE_RADIAL, width=1008, height=756, num_params=4'>}

In [ ]:
import numpy as np
from pathlib import Path
image_path = "/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/1000Hour_placeno16"
txt = open(Path(image_path) / "localize_result_upper_p16.txt", 'r').read().split("\n")[:-1]

image_idx = 1
for tx in txt:
    tx = tx.split(",")
    image_name = tx[0]
    t = np.array(tx[1:4])
    if t[0] == 'ERROR': continue
    print(t)
    q = np.array(tx[4:])
    try:
        image_sfm = recon2.images[image_idx]
    except Exception as e:
        print(e)
    image_sfm.tvec = t
    image_sfm.qvec = q
    image_sfm.name = image_name
    image_sfm.image_id = image_idx
    # image_sfm.camera_id = 2
    print(image_sfm)
    # recon2.add_image(image_sfm)
    image_idx += 1
    # break

In [12]:
image_idx

1918

In [13]:
image_sfm

<Image 'image_id=1917, camera_id=1, name="sr1-101_2496_16_0.jpg", triangulated=828/3208'>

In [15]:
recon2.images.__len__()

198

In [16]:
image_sfm

<Image 'image_id=1917, camera_id=1, name="sr1-101_2496_16_0.jpg", triangulated=828/3208'>

In [18]:
recon2.write('/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/1000Hour_outputs/bld4-upper-p16')

In [1]:
recon2

NameError: name 'recon2' is not defined